In [1]:
import pandas as pd

In [3]:
#this class is merely for encapsulation.
#this is how data of the dataset is stores in the objects 
#of this class.
class DataSet:
    #name of the dataset
    def __init__(self, name):
        self.name = name
    
    def set_feature(self, features):
        self.features = features
    
    def set_classes(self, classes):
        self.classes = classes
    
    def get_classes(self):
        return self.classes
        
    def set_org_data(self, org_data):
        self.org_data = org_data
        
    def get_org_data(self):
        return self.org_data
    
    # not writing all getters, setters, because
    # they are not required in python (i didnt know that before)

In [4]:
#Class reads the data file and the names file
#to load inti dataset object
class Dataset_Reader:
    
    def __init__(self):
        self.Path = 'C:\\Users\\nitas\\Downloads\\Machine Learning 2024\\MLProject2\\'
    
    #read the data from data file and fill the Datset class object
    def read_data_file(self, ds):
        if(ds.name == 'forestfires'):
            df = pd.read_csv(self.Path+ds.name+'.data', delimiter=',')
        else:    
            df = pd.read_csv(self.Path+ds.name+'.data', delimiter=',', header=None)

        ds.set_org_data(df)
    
    
    #read name file, naming the features, just to fill
    #for results, not mendatory function
    def read_attributes(self, ds):
        
        start_string = 'Attribute Information'
        stop_string = 'Missing Attribute Values'
        isPrint = False
        
        #read section, where the file has information 
        #Attribute Information
        with open(Path+ds.name+'.names', 'r') as file:
            for line in file:
                if start_string in line:
                    isPrint = True
                    #continue
                    
                if stop_string in line:
                    isPrint = False
                    break
                        
                if isPrint:
                    print(line.strip())
                        

In [5]:
#Pre Process Class, for data cleaning, 
#checking for any abnormalities, pre processes the data 
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler


class PreProcess:
    
    def __init__(self, ds):
        
        #set other values of the Dataset object    
        self.set_Dataset_features(ds)
        
        #min max scaling for the regression datasets
        if(ds.type == 'Regression'):
            self.min_max_scaling(ds)
        if(ds.type == 'Classification'):
            self.min_max_scaling_class(ds)
        #now first step is to do one-hot encoding
        self.one_hot_encoding(ds)
        #Now sort the data, if it is not sorted
        self.sort_data(ds)
        #After this step, we will use only sorted data
        self.missing_values(ds)
    
    
    #Zscore Normalization 
    def zscore_normalization(self, ds):
        scaler = StandardScaler()
        numeric_cols = ds.org_data.select_dtypes(include=['float64', 'int64']).columns
        ds.org_data[numeric_cols] = scaler.fit_transform(ds.org_data[numeric_cols])
    
    
    def zscore_normalization_class(self, ds):
        scaler = StandardScaler()
        numeric_cols = ds.org_data.iloc[:, :-1].select_dtypes(include=['float64', 'int64']).columns
        ds.org_data[numeric_cols] = scaler.fit_transform(ds.org_data[numeric_cols])
        
    #Min max scaling 
    def min_max_scaling(self, ds):
        scaler = MinMaxScaler()
        #print(ds.org_data.head())
        # Apply scaling only to numeric columns
        numeric_cols = ds.org_data.select_dtypes(include=['float64', 'int64']).columns
        #print(numeric_cols)
        ds.org_data[numeric_cols] = scaler.fit_transform(ds.org_data[numeric_cols]) 
        #print('After applying Min Max Scaling:' ,ds.org_data.head())
        #Min max scaling 
        
    #for classification, we will exclude the class column    
    def min_max_scaling_class(self, ds):
        scaler = MinMaxScaler()
        #print(ds.org_data.head())
        # Apply scaling only to numeric columns
        numeric_cols = ds.org_data.iloc[:, :-1].select_dtypes(include=['float64', 'int64']).columns
        #print(numeric_cols)
        ds.org_data[numeric_cols] = scaler.fit_transform(ds.org_data[numeric_cols]) 
        #print('After applying Min Max Scaling:' ,ds.org_data.head())
    
    #This function sets the no of features and class names
    #to the Dataset class object
    def set_Dataset_features(self, ds):
        #set class names
        last_column_unique = ds.org_data.iloc[:, -1].unique()
        ds.set_classes(last_column_unique)
        #print(ds.get_classes())
        
        #set no of features, because one column is classes
        ds.feature_count = ds.org_data.shape[1] -1
        #print(ds.feature_count)
    
                
    #splitting data into 10 parts, Here we will
    #implement stratified 10 folds
    def split_data_regression(self,ds):
        
        # first we need to make 10 datasets of consectutive points
        #print('\n Printing the results of 10 folds')
        datagroupSize = len(ds.sorted_data) // 10  
        print('Fold Size', datagroupSize)
        dataGroup = []
        
        #reusing this part from my first project
        for i in range(10):
            start = i * datagroupSize
            end = start + datagroupSize
            if i == 9:  
                dataGroup.append(ds.sorted_data.iloc[start:])
            else:
                dataGroup.append(ds.sorted_data.iloc[start:end])
        
        
        #once we have datasets of sorted data, now we will make folds
        #with one point from each fold.
        folds = [[] for _ in range(10)]
        
        max_group_size = max(len(group) for group in dataGroup)
        print('Max Fold Size: ',max_group_size)
        
        # For each index up to the maximum group size
        for i in range(max_group_size):
            # Loop over the 10 groups
            for j in range(10):
                #print('fold:', j, 'datagroup :' ,j, 'value',  dataGroup[j].iloc[i, -1])
                # Check if the current group has enough data for the current index
                if i < len(dataGroup[j]):  
                    # Append the i-th example from the j-th group into the j-th fold
                    folds[j].append(dataGroup[j].iloc[i])

                    
        #converting folds into Dataframes
        final_folds = folds = [pd.DataFrame(fold) for fold in folds]
        ds.ten_folds = final_folds
        
        #for i in range(10):
        #    print(f'Fold {i + 1}:')
        #    print(final_folds[i].tail())
        
        #print("Ten Fold Prints")
        #for i, fold in enumerate(final_folds):
        #    print(f'Fold {i+1} size: {len(fold)}')
        #    print(fold.iloc[:, -1].value_counts())
    
    
    #splitting data into 10 parts, Here we will
    #implement stratified 10 folds for classification
    def split_data_classification(self,ds):
        
        #In this we first need to split data accordign to classes
        #Since we want the ratio of classes to be same in each fold
        classwise_data = {}
        
        for cls in ds.classes:
            classwise_data[cls] = ds.sorted_data[ds.sorted_data.iloc[:, -1] == cls]
                    
        # Now we will be creating the folds containing data from 
        # each fold of class, making the ratio same with original data
        folds = [[] for _ in range(10)]
        
        for cls, cls_data in classwise_data.items():
            
            #first divide the classwise data into 10 parts.
            clsPartsforFolds = len(cls_data) //10
            clsPartsRemain = len(cls_data) % 10
            
            #print('Number of class data per fold',clsPartsforFolds)
            #print('Remaining data left in class',clsPartsRemain)
            #put these values in all folds
            start = 0
            for i in range(10):
                if clsPartsRemain != 0:
                    # Add 1 to handle remaining data
                    end = start + clsPartsforFolds + 1  
                    clsPartsRemain -= 1
                else:
                    end = start + clsPartsforFolds
            
                # Append class data slice to the respective fold
                folds[i].extend(cls_data.iloc[start:end].values.tolist())
                start = end
            
        
        #converting folds into Dataframes
        final_folds = [pd.DataFrame(fold, columns=ds.sorted_data.columns) for fold in folds]    
        ds.ten_folds = final_folds
        
        #This section is just to check if the fold created are in the same ratio
        #for i in range(10):
        #    print(f'Fold {i + 1}:')
        #    print(final_folds[i].tail())
        
        #for i, fold in enumerate(final_folds):
        #    print(f'Fold {i+1} size: {len(fold)}')
        #    print(fold.iloc[:, -1].value_counts())
        
    
    #added the function to see, if we will need it for any dataset
    def one_hot_encoding(self, ds):
        #this has first coloumn of categorical data, F, M and I
        if(ds.name == 'abalone'):
            #save class column and drop it
            class_column = ds.org_data.iloc[:, -1]  
            ds.org_data = ds.org_data.drop(ds.org_data.columns[-1], axis=1)
            
            #perform one hot encoding    
            first_column_name = ds.org_data.columns[0]
            one_hot_encoded = pd.get_dummies(ds.org_data[first_column_name], prefix=first_column_name)
            ds.org_data = pd.concat([ds.org_data.drop(columns=[first_column_name]), one_hot_encoded], axis=1)
            #append the saved class column to the end again
            ds.org_data = pd.concat([ds.org_data, class_column], axis=1)
        
        #this dataset has first column of manufacturer, and second column of 
        #model name, which has unique values, so dropping it.
        if(ds.name == 'machine'):
            
            #here i am dropping the last colum, because class / result is 
            ds.org_data = ds.org_data.drop(ds.org_data.columns[-1], axis=1)
            #save the class column and then drop it   
            class_column = ds.org_data.iloc[:, -1]
            ds.org_data = ds.org_data.drop(ds.org_data.columns[-1], axis=1)
            
            #drop the second column, which is model number
            ds.org_data = ds.org_data.drop(ds.org_data.columns[1], axis=1)
            first_column_name = ds.org_data.columns[0]
            one_hot_encoded = pd.get_dummies(ds.org_data[first_column_name], prefix=first_column_name)
            ds.org_data = pd.concat([ds.org_data.drop(columns=[first_column_name]), one_hot_encoded], axis=1)
            
            #append the saved class column to the end again
            ds.org_data = pd.concat([ds.org_data, class_column], axis=1)
        
        #first column is months, second column is days
        if ds.name == 'forestfires':
            
            #tranform the last column to log 
            #*****This is still not helping, we might need normalization*****
            ds.org_data['area'] = np.log(ds.org_data['area'] +  1)
            
            
            #save the class column and then drop it, 
            class_column = ds.org_data.iloc[:, -1]
            ds.org_data = ds.org_data.drop(ds.org_data.columns[-1], axis=1)
            
            first_column_name = ds.org_data.columns[2]
            second_column_name = ds.org_data.columns[3]
            one_hot_encoded = pd.get_dummies(ds.org_data[[first_column_name, second_column_name]], 
                                             prefix=[first_column_name, second_column_name])
                                    
            ds.org_data = pd.concat([ds.org_data.drop(columns=[first_column_name, second_column_name]), 
                                     one_hot_encoded], axis=1)
            
            #append the saved class column to the end again
            ds.org_data = pd.concat([ds.org_data, class_column], axis=1)
        
        #print("One hot Encoding Step, Resulted data shown below")
        #print(ds.org_data.head())
        
    #First we need to sort the data, if not sorted according to the 
    #classes, if data is already sorted, then original values will remain
    #same without change
    def sort_data(self, ds):
        ds.sorted_data = ds.org_data.sort_values(by=ds.org_data.columns[-1])
        #and then reset the index
        ds.sorted_data = ds.sorted_data.reset_index(drop=True)
        
        
    # this function handles missing values, for this project
    # forest fires has no missing values
    # Abalone has no missing values
    # machine has no missing values
    # glass has no missing values
    # Soybean has no missing values
    # breastCancer has missing values
    def missing_values(self, ds):      
        # Check if any missing values exist, using '?' and null values
        ds.sorted_data.replace('?', np.nan, inplace=True)
        has_missing_values = ds.sorted_data.isnull().values.any()
        missing_values_count = ds.sorted_data.isnull().sum().sum()
        
        if (has_missing_values):
            print("This dataset has missing values")
            if missing_values_count > 1:
                # It has missing values  
                #print("This dataset has more missing values")
                rows_with_missing = dds.sorted_data[ds.sorted_data.isnull().any(axis=1)].index
                #print("Rows with missing values before filling:")
                #print(ds.ds.sorted_data.loc[rows_with_missing])
                
                if(ds.name == 'breast-cancer-wisconsin'):
                    #print("Inside the median if")
                    for column in ds.sorted_data.columns[:-1]:
                        median_value = ds.sorted_data[column].median()
                        # Filling the missing values
                        ds.sorted_data[column].fillna(median_value, inplace=True)
    
                #print("Rows with missing values after filling:")
                #print(ds.sorted_data.loc[rows_with_missing])
            else:
                ds.sorted_data.dropna(inplace=True)
                print("Dropped the row")

                
    #This one function will work for classification    
    def split_tuning_data_classification(self, ds):
        print('Spliting the data into tunning and remaining')
        # Backup sorted data
        ds.sorted_data_bk = ds.sorted_data.copy()
    
        # Initialize empty lists for tuning and remaining data
        tuning_data = []
        remaining_data = []

        # Separate the data into classes
        for cls in ds.classes:
            #print(cls)
            classwise_data = ds.sorted_data[ds.sorted_data.iloc[:, -1] == cls]

            # Shuffle the data
            classwise_data = classwise_data.sample(frac=1, random_state=42).reset_index(drop=True)
        
            # Extract 10% tuning set
            num_tuning_samples = max(1, int(len(classwise_data) * 0.1))  
            #print('number of samples from this class',num_tuning_samples )
        
            # Append tuning set and remaining data
            tuning_data.append(classwise_data.iloc[:num_tuning_samples])
            remaining_data.append(classwise_data.iloc[num_tuning_samples:])
    
        # Concatenate tuning and remaining data
        ds.tuning_data = pd.concat(tuning_data).reset_index(drop=True)
        ds.sorted_data = pd.concat(remaining_data).reset_index(drop=True)
        
        #print('Length of tuning data:',len(ds.tuning_data) )
        #print('Length of remaining data:',len(ds.sorted_data) )
        #print('Length of original data:',len(ds.sorted_data_bk) )

        
    def split_tuning_data_regression(self, ds):
        # Backup sorted data
        
        print('\n Spliting the data into tunning and remaining')
        tuning_fraction = 0.1
        n_bins = 10
        ds.sorted_data_bk = ds.sorted_data.copy()

        # Bin the target variable into quantiles (or any number of bins)
        ds.sorted_data['bins'] = pd.qcut(ds.sorted_data.iloc[:, -1], q=n_bins, labels=False, duplicates='drop')

        # Initialize empty lists for tuning and remaining data
        tuning_data = []
        remaining_data = []

        # Separate the data into bins (which act as pseudo-classes for stratification)
        for bin_value in ds.sorted_data['bins'].unique():
            bin_data = ds.sorted_data[ds.sorted_data['bins'] == bin_value]

            # Shuffle the data
            bin_data = bin_data.sample(frac=1, random_state=42).reset_index(drop=True)

            # Extract the specified fraction (10% by default) for the tuning set
            num_tuning_samples = int(len(bin_data) * tuning_fraction)

            # Append tuning set and remaining data
            tuning_data.append(bin_data.iloc[:num_tuning_samples])
            remaining_data.append(bin_data.iloc[num_tuning_samples:])

        # Concatenate tuning and remaining data
        ds.tuning_data = pd.concat(tuning_data).reset_index(drop=True)
        ds.sorted_data = pd.concat(remaining_data).reset_index(drop=True)

        # Drop the 'bins' column, as it's not needed anymore
        ds.sorted_data.drop(columns=['bins'], inplace=True)
        ds.tuning_data.drop(columns=['bins'], inplace=True)

        # Check lengths
        #print('Length of tuning data:', len(ds.tuning_data))
        #print('Length of remaining data:', len(ds.sorted_data))
        #print('Length of original data:', len(ds.sorted_data_bk))    
        
        
        

In [6]:
from abc import ABC, abstractmethod

class Layer(ABC): 
    def __init__(self):
        self.input = None
        self.output = None

    @abstractmethod
    # This is to enforces subclass to implement this method
    def forward_propagation(self, inputData):
        pass

    @abstractmethod
    def backward_propagation(self, gradient, learning_rate):
        pass


In [7]:
#inheritence from abstract class layer
class FCHiddenLayer(Layer):
    
    #the help for initiazing the input randomly has been taken from 
    #https://towardsdatascience.com/math-neural-network-from-scratch-in-python-d6da9f29ce65
    def __init__(self, inputSize, outputSize):
        # Random initialization of weights and bias
        self.weights = np.random.rand(inputSize, outputSize) - 0.5
        self.bias = np.random.rand(1, outputSize) - 0.5

    # fully connected layer implementation of forward propagation
    # that is input will be multiplied with weights    
    def forward_propagation(self, inputData):
        # Store the input for backpropagation
        self.input = inputData  
        
        # (inputData * weights) + bias (broadcasted across all rows)
        outputData = np.dot(self.input, self.weights) + self.bias
        self.output = outputData
        
        
        #print('\nForward Propagarion of Hidden Layer')
        #print("Input shape:", self.input.shape)  
        #print("output shape:", self.output.shape)  
        #print("Weights shape:", self.weights.shape)
        return self.output

    def backward_propagation(self, gradient, learning_rate):
        
        gradient = np.clip(gradient, -100, 100)
        # Ensure the input has 2 dimensions (batch_size, inputSize)
        if len(self.input.shape) == 1:
            self.input = self.input.reshape(1, -1)

        #print('\nBack Propagarion of Hidden Layer')
        #print("Input shape:", self.input.shape)  
        #print("Gradient shape:", gradient.shape)  
        #print("Weights shape:", self.weights.shape)    
                    
        # Calculate gradients for weights and inputError (backpropagated error)
        inputError = np.dot(gradient, self.weights.T)  
        weightsError = np.dot(self.input.T, gradient)  
        
        # Update weights and bias
        self.weights -= learning_rate * weightsError
        self.bias -= learning_rate * np.sum(gradient, axis=0, keepdims=True)
        return inputError


In [8]:

#inheritence from abstract class layer
class ActivationLayer(Layer):
    
    # set the  activation function (e.g., sigmoid or tanh)
    # set the derivative of the activation function
    def __init__(self, activation, activation_derivative):
        self.activation = activation
        self.activation_derivative = activation_derivative

    # here we will not multiply the weights and add bias
    # but rather we will use activation function on the data
    # returned from fully connected layer
    def forward_propagation(self, inputData):

        self.input = inputData  
        self.output = self.activation(self.input)  # Apply activation to the entire batch
        
        #print('\nForward Propagarion of Activation Layer')
        #print("Input shape:", self.input.shape)  
        #print("Output shape:", self.output.shape)
        
        return self.output

    def backward_propagation(self, gradient, learning_rate):
        gradient = np.clip(gradient, -100, 100)
        # Apply the derivative of the activation function element-wise
        output = self.activation_derivative(self.input) * gradient
        
        #print('\nBackward Propagarion of Activation Layer')
        #print("Gradient shape:", gradient.shape)  
        #print("Output shape:", output.shape)
        
        return output            
    
    # Activation functions
    @staticmethod
    def tanh(x):
        return np.tanh(x)

    @staticmethod
    def tanh_derivative(x):
        return 1 - np.tanh(x)**2
    
    @staticmethod
    def sigmoid(x):
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

    # Derivatives of the activation functions
    @staticmethod
    def sigmoid_derivative(x):
        sig = ActivationLayer.sigmoid(x)
        return np.clip(sig * (1 - sig), 1e-7, 1 - 1e-7)
    
        
    # Linear activation for regression (identity function)
    @staticmethod
    def linear(x):
        return x
    
    @staticmethod
    def linear_derivative(x):
        return 1
    
    @staticmethod
    def softmax(x):
        exp_vals = np.exp(x - np.max(x))  # Subtract max for numerical stability
        return exp_vals / np.sum(exp_vals)

    @staticmethod
    def softmax_derivative(x):
        # Assuming we already have softmax output x, softmax derivative is:
        return x * (1 - x)
    

In [9]:
import numpy as np


class NeuralNetwork:
    
    def __init__(self, lossFunction=None, lossFunctionDerivative=None):
        self.layers = []
        self.loss = lossFunction
        self.loss_derivative = lossFunctionDerivative

    # this will be used for all layers, activation, hidden and output    
    def addLayers(self, layer):
        self.layers.append(layer)
        
    def predict(self, test_features):
        rows = len(test_features)
        result = []

        for i in range(rows):
            output = test_features[i]
            #this loop will call the forward propagation function of 
            #each layer, hidden, activation and output
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result
    
    
    #Dealing with mini batches. Also added early stopping on convergence
    def train(self, train_features, train_class, epochs, learning_rate, batch_size=32, patience=5):
        no_of_samples = len(train_features)
        best_loss = float('inf')  # Initialize best_loss to a large value
        patience_counter = 0      # Counter to track patience

        for epoch in range(epochs):
            err = 0

            # Shuffle the data
            indices = np.arange(no_of_samples)
            np.random.shuffle(indices)
            x_train = train_features[indices]
            y_train = train_class[indices]

            # Iterate over batches
            for j in range(0, no_of_samples, batch_size):
                end = min(j + batch_size, no_of_samples)
                x_batch = x_train[j:end]
                y_batch = y_train[j:end]

                # Forward propagation for the whole batch
                output = x_batch
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # Calculate the error/loss for the batch
                err += self.loss(y_batch, output)

                # Find the gradient and backpropagate
                error = self.loss_derivative(y_batch, output)
                #print(f"Initial error shape: {error.shape}")
                
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            err /= no_of_samples  # Average error over the entire dataset
            print(f'Epoch {epoch+1}/{epochs}   Error={err:.6f}')

            # Early stopping condition
            if err < best_loss:
                best_loss = err
                patience_counter = 0  # Reset patience counter if model improves
            else:
                patience_counter += 1

            if patience_counter >= patience:
                print(f'Stopping early at epoch {epoch+1} due to no improvement.')
                break

    # this the the loss function part, 
    # mse for regression 

    @staticmethod
    def mse(actual_value, pred_value):
        return np.mean(np.power(actual_value-pred_value, 2))
    
    @staticmethod
    def mse_derivative(actual_value, pred_value):
        if actual_value.shape != pred_value.shape:
            actual_value = actual_value.reshape(pred_value.shape)

        #print(f"After reshaping: actual_value shape: {actual_value.shape}, pred_value shape: {pred_value.shape}")

        return 2 * (pred_value - actual_value) / actual_value.shape[0]

    # for classification (binary)
    @staticmethod
    def binary_crossentropy(actual_value, pred_value):
        # Avoid log(0) by clipping values
        pred_value = np.clip(pred_value, 1e-12, 1 - 1e-12)
        return -np.mean(actual_value * np.log(pred_value) + (1 - actual_value) * np.log(1 - pred_value))

    @staticmethod
    def binary_crossentropy_derivative(actual_value, pred_value):
        # Avoid division by zero
        pred_value = np.clip(pred_value, 1e-12, 1 - 1e-12)
        return (pred_value - actual_value) / (pred_value * (1 - pred_value) * actual_value.size)


    # for multiclass classification
    @staticmethod
    def categorical_crossentropy(actual_value, pred_value):
        # Avoid log(0) by clipping values
        pred_value = np.clip(pred_value, 1e-12, 1 - 1e-12)
        return -np.mean(np.sum(actual_value * np.log(pred_value), axis=1))
    
    @staticmethod
    def categorical_crossentropy_derivative(y_true, y_pred):    
        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clipping to prevent log(0)

        return (y_pred - y_true) / y_true.shape[0]

In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

class ClassificationNet:
    def __init__(self):
        self.net = None  # Initialize the neural network
    
    def tuneLearningRate(self, ds, no_of_layers):
        best_accuracy = 0
        best_lr = None

        # Use tuning data for testing and the rest for training
        ds.test_data = ds.tuning_data
        ds.train_data = pd.concat(ds.ten_folds[:9])

        # Split features and labels
        train_features = ds.train_data.iloc[:, :-1]
        train_cls = ds.train_data.iloc[:, -1]
        test_features = ds.test_data.iloc[:, :-1]
        test_cls = ds.test_data.iloc[:, -1]

        x_train = train_features.to_numpy()
        y_train = train_cls.to_numpy()
        x_test = test_features.to_numpy()
        y_test = test_cls.to_numpy()
        input_size = train_features.shape[1]
        
        #print('Dataset: ', ds.name, ' InputSize:', input_size, 'No of classes:', ds.classes)
        
        # One-hot encode for labels
        encoder = OneHotEncoder(sparse_output=False)
        y_train = encoder.fit_transform(train_cls.values.reshape(-1, 1))  
        y_test = encoder.transform(test_cls.values.reshape(-1, 1))  
        

        # Iterate over learning rates
        for lr in [0.001, 0.01, 0.1]:
            print(f"Testing learning rate: {lr}")
            # make neural network
            self.makeNeuralNetwork(ds, no_of_layers, input_size)
            # train neural network
            self.net.train(x_train, y_train, epochs=100, learning_rate=lr)
            # get Predictions
            predictions = self.net.predict(x_test)
            
            predictions = np.vstack(predictions) 
            
            if ds.name == 'breast-cancer-wisconsin':
                predicted_classes = (predictions > 0.5).astype(int)
            else:
                predicted_classes = np.argmax(predictions, axis=1)
                    
            true_classes = np.argmax(y_test, axis=1)
            
            #print("Predictions:", predictions[:5])  # Inspect predictions
            #print("Predicted Classes:", predicted_classes[:5])  # Inspect class predictions
            #print("True Classes:", true_classes[:5])  # Inspect true class labels
            
            
            #results = pd.DataFrame({'Actual Class': true_classes, 'Predicted Class': predicted_classes})
            #print(results)    

            accuracy = np.sum(predicted_classes == true_classes) / len(true_classes)
            print(f"learning rate: {lr}, with accuracy: {accuracy}")
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_lr = lr

        print(f"Best learning rate: {best_lr}, with accuracy: {best_accuracy}")
        self.learning_rate = best_lr

        
    def tuneNoofNeurons(self,ds, no_of_layers):
        
        best_accuracy = 0
        best_lr = None

        # Use tuning data for testing and the rest for training
        ds.test_data = ds.tuning_data
        ds.train_data = pd.concat(ds.ten_folds[:9])

        # Split features and labels
        train_features = ds.train_data.iloc[:, :-1]
        train_cls = ds.train_data.iloc[:, -1]
        test_features = ds.test_data.iloc[:, :-1]
        test_cls = ds.test_data.iloc[:, -1]

        x_train = train_features.to_numpy()
        y_train = train_cls.to_numpy()
        x_test = test_features.to_numpy()
        y_test = test_cls.to_numpy()
        input_size = train_features.shape[1]
        
        #print('Dataset: ', ds.name, ' InputSize:', input_size, 'No of classes:', ds.classes)
        
        # One-hot encode for labels
        encoder = OneHotEncoder(sparse_output=False)
        y_train = encoder.fit_transform(train_cls.values.reshape(-1, 1))  
        y_test = encoder.transform(test_cls.values.reshape(-1, 1))  
        
        
        
        
        
    def makeNeuralNetwork(self, ds, no_of_layers,input_size):
        
        de_instance =  
        
        if ds.name == 'breast-cancer-wisconsin':
            self.net = NeuralNetwork(NeuralNetwork.binary_crossentropy, 
                                     NeuralNetwork.binary_crossentropy_derivative)
            output_activation = ActivationLayer.sigmoid
            output_activation_derivative = ActivationLayer.sigmoid_derivative
            output_size = 1  # Binary classification

        else:
            self.net = NeuralNetwork(NeuralNetwork.categorical_crossentropy, 
                                     NeuralNetwork.categorical_crossentropy_derivative)
            output_activation = ActivationLayer.softmax
            output_activation_derivative = ActivationLayer.softmax_derivative
            output_size = len(ds.classes) 
          
        if no_of_layers == 0:
            
            self.net.addLayers(FCHiddenLayer(input_size, output_size))
            self.net.addLayers(ActivationLayer(output_activation, output_activation_derivative))
        
        elif no_of_layers == 1:
            self.net.addLayers(FCHiddenLayer(input_size, 3))
            self.net.addLayers(ActivationLayer(ActivationLayer.tanh, ActivationLayer.tanh_derivative))
            self.net.addLayers(FCHiddenLayer(3, output_size))
            self.net.addLayers(ActivationLayer(output_activation, output_activation_derivative))

        elif no_of_layers == 2:
            self.net.addLayers(FCHiddenLayer(input_size, 3))
            self.net.addLayers(ActivationLayer(ActivationLayer.tanh, ActivationLayer.tanh_derivative))
            self.net.addLayers(FCHiddenLayer(3, 3))
            self.net.addLayers(ActivationLayer(ActivationLayer.tanh, ActivationLayer.tanh_derivative))
            self.net.addLayers(FCHiddenLayer(3, output_size))
            self.net.addLayers(ActivationLayer(output_activation, output_activation_derivative))

    def mainFunction(self, ds, no_of_layers):
        fold_accuracies = []

        for i in range(len(ds.ten_folds)):  # Assuming len(ds.ten_folds) is 10 for 10-fold CV
            # One fold for testing, the rest for training
            print(f'Fold: {i}')
            ds.test_data = ds.ten_folds[i]
            ds.train_data = pd.concat([ds.ten_folds[j] for j in range(10) if j != i])

            # Split features and labels
            train_features = ds.train_data.iloc[:, :-1]
            train_cls = ds.train_data.iloc[:, -1]
            test_features = ds.test_data.iloc[:, :-1]
            test_cls = ds.test_data.iloc[:, -1]

            x_train = train_features.to_numpy()
            y_train = train_cls.to_numpy()
            x_test = test_features.to_numpy()
            y_test = test_cls.to_numpy()
            input_size = train_features.shape[1]
            
            
            encoder = OneHotEncoder(sparse_output=False)
            y_train = encoder.fit_transform(train_cls.values.reshape(-1, 1))  
            y_test = encoder.transform(test_cls.values.reshape(-1, 1))  

            # Train the neural network for this fold
            self.makeNeuralNetwork(ds, no_of_layers, input_size)
            self.net.train(x_train, y_train, epochs=100, learning_rate=self.learning_rate)

            predictions = self.net.predict(x_test)
            predictions = np.vstack(predictions)
            
            # Calculate accuracy
            if ds.name == 'breast-cancer-wisconsin':
                predicted_classes = (predictions > 0.5).astype(int)
            else:
                predicted_classes = np.argmax(predictions, axis=1)
                
            #Convert one hot encoding back    
            true_classes = np.argmax(y_test, axis=1)
            
            results = pd.DataFrame({'Actual Class': true_classes, 'Predicted Class': predicted_classes})
            print(results)
            
            
            accuracy = np.sum(predicted_classes == true_classes) / len(true_classes)

            fold_accuracies.append(accuracy)
            print(f'Fold: {i}, Accuracy: {accuracy}')

        # After all folds, calculate and print the mean accuracy
        mean_accuracy = np.mean(fold_accuracies)
        print(f'Mean Accuracy across all folds: {mean_accuracy}')


In [18]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

class RegressionNet:
    def __init__(self):
        # Initialize the neural network
        self.net = None  
    
    def tuneLearningRate(self, ds, no_of_layers):
        
        lowest_mse = np.inf  
        best_lr = None

        # Use tuning data for testing and the rest for training
        ds.test_data = ds.tuning_data
        ds.train_data = pd.concat(ds.ten_folds[:9])

        # Split features and labels
        train_features = ds.train_data.iloc[:, :-1]
        train_cls = ds.train_data.iloc[:, -1]
        test_features = ds.test_data.iloc[:, :-1]
        test_cls = ds.test_data.iloc[:, -1]

        x_train = train_features.to_numpy()
        y_train = train_cls.to_numpy()
        x_test = test_features.to_numpy()
        y_test = test_cls.to_numpy()
        input_size = train_features.shape[1]

        print('Dataset: ', ds.name, ' InputSize:', input_size)

        # Iterate over learning rates
        for lr in [0.001, 0.01, 0.1]:
            print(f"Testing learning rate: {lr}")
            # Make neural network
            self.makeNeuralNetwork(ds, no_of_layers, input_size)
            # Train neural network
            self.net.train(x_train, y_train, epochs=100, learning_rate=lr)
            # Get predictions
            predictions = self.net.predict(x_test)

            results = pd.DataFrame({'Actual Class': y_test, 'Predicted Class': predictions})

            # Calculate the error (MSE for regression tasks)
            mse = self.mse(results['Actual Class'], results['Predicted Class'])
            print(f'Learning Rate: {lr}, with error: {mse}')

            # Update the best learning rate if the current MSE is lower
            if lowest_mse > mse:
                lowest_mse = mse  # Consistent variable usage
                best_lr = lr

        print(f"Best learning rate: {best_lr}, with error: {lowest_mse}")
        self.learning_rate = best_lr


        
    def makeNeuralNetwork(self, ds, no_of_layers,input_size):

        self.net = NeuralNetwork(NeuralNetwork.mse, NeuralNetwork.mse_derivative)
        output_activation = ActivationLayer.linear
        output_activation_derivative = ActivationLayer.linear_derivative
        output_size = 1         
          
        if no_of_layers == 0:
            self.net.addLayers(FCHiddenLayer(input_size, output_size))
            self.net.addLayers(ActivationLayer(output_activation, output_activation_derivative))
        
        elif no_of_layers == 1:
            self.net.addLayers(FCHiddenLayer(input_size, 3))
            self.net.addLayers(ActivationLayer(ActivationLayer.tanh, ActivationLayer.tanh_derivative))
            self.net.addLayers(FCHiddenLayer(3, output_size))
            self.net.addLayers(ActivationLayer(output_activation, output_activation_derivative))

        elif no_of_layers == 2:
            self.net.addLayers(FCHiddenLayer(input_size, 3))
            self.net.addLayers(ActivationLayer(ActivationLayer.tanh, ActivationLayer.tanh_derivative))
            self.net.addLayers(FCHiddenLayer(3, 3))
            self.net.addLayers(ActivationLayer(ActivationLayer.tanh, ActivationLayer.tanh_derivative))
            self.net.addLayers(FCHiddenLayer(3, output_size))
            self.net.addLayers(ActivationLayer(output_activation, output_activation_derivative))

    def mainFunction(self, ds, no_of_layers):
        fold_errors = []

        for i in range(len(ds.ten_folds)):  
            print(f'Fold: {i}')

            # One fold for testing, the rest for training
            ds.test_data = ds.ten_folds[i]
            ds.train_data = pd.concat([ds.ten_folds[j] for j in range(len(ds.ten_folds)) if j != i])

            # Split features and labels
            train_features = ds.train_data.iloc[:, :-1]
            train_cls = ds.train_data.iloc[:, -1]
            test_features = ds.test_data.iloc[:, :-1]
            test_cls = ds.test_data.iloc[:, -1]

            x_train = train_features.to_numpy()
            y_train = train_cls.to_numpy()
            x_test = test_features.to_numpy()
            y_test = test_cls.to_numpy()
            input_size = train_features.shape[1]

            # Train the neural network for this fold
            self.makeNeuralNetwork(ds, no_of_layers, input_size)
            self.net.train(x_train, y_train, epochs=100, learning_rate=self.learning_rate)

            predictions = self.net.predict(x_test)
            results = pd.DataFrame({'Actual Class': y_test, 'Predicted Class': predictions})

            # Calculate the error (MSE for regression tasks)
            mse = self.mse(results['Actual Class'], results['Predicted Class'])
            print(f'Fold: {i}, Mean Squared Error: {mse}')

            # Store fold error
            fold_errors.append(mse)

            print(f'Fold: {i}, Error: {mse}')

        # After all folds, calculate and print the mean error
        mean_errors = np.mean(fold_errors)
        print(f'Mean Error across all folds: {mean_errors}')

        
    def mse(self, actual_value, pred_value):
        return np.mean(np.power(actual_value - pred_value, 2))
        
    


In [19]:
#Main driver class calls all other classes  

class Driver:
    def __init__(self):
        self.DSNames_classification = 'glass', 'soybean-small', 'breast-cancer-wisconsin'
        self.DSNames_regression = 'abalone', 'machine', 'forestfires'
    
    #classification
    def main_classification(self):
        
        #Preprocessing part
        #change the dataset number here from 0 to 2
        ds = DataSet(self.DSNames_classification[0])
        ds.type = 'Classification'
        ds_reader = Dataset_Reader()
        ds_reader.read_data_file(ds)
            
        pp = PreProcess(ds)
        pp.split_tuning_data_classification(ds)
        pp.split_data_classification(ds)
        
        cn = ClassificationNet()
        cn.tuneLearningRate(ds, 2)
        cn.mainFunction(ds, 0)
        cn.mainFunction(ds, 1)
        cn.mainFunction(ds, 2)
        
       
        
    #Regression
    def main_regression(self):
        #change the dataset number here from 0 to 2
        ds = DataSet(self.DSNames_regression[1])
        ds.type = 'Regression'
        ds_reader = Dataset_Reader()
        ds_reader.read_data_file(ds)
        
        pp = PreProcess(ds)
        pp.split_tuning_data_regression(ds)
        pp.split_data_regression(ds)
        
        rn = RegressionNet()
        rn.tuneLearningRate(ds, 2)
        rn.mainFunction(ds, 0)
        
        

        
        
#calling the driver class here.    
dr = Driver()
dr.main_classification() 
#dr.main_regression()
        

Spliting the data into tunning and remaining
Testing learning rate: 0.001
Epoch 1/100   Error=0.173796
Epoch 2/100   Error=0.173718
Epoch 3/100   Error=0.173891
Epoch 4/100   Error=0.174010
Epoch 5/100   Error=0.173977
Epoch 6/100   Error=0.173985
Epoch 7/100   Error=0.174078
Stopping early at epoch 7 due to no improvement.
learning rate: 0.001, with accuracy: 0.05263157894736842
Testing learning rate: 0.01
Epoch 1/100   Error=0.175991
Epoch 2/100   Error=0.176171
Epoch 3/100   Error=0.175975
Epoch 4/100   Error=0.176385
Epoch 5/100   Error=0.176572
Epoch 6/100   Error=0.177148
Epoch 7/100   Error=0.177010
Epoch 8/100   Error=0.176560
Stopping early at epoch 8 due to no improvement.
learning rate: 0.01, with accuracy: 0.05263157894736842
Testing learning rate: 0.1
Epoch 1/100   Error=0.177633
Epoch 2/100   Error=0.183806
Epoch 3/100   Error=0.260767
Epoch 4/100   Error=0.432420
Epoch 5/100   Error=0.431123
Epoch 6/100   Error=0.447429
Stopping early at epoch 6 due to no improvement.
le